In [1]:
import pandas as pd
import os
import awswrangler as wr
env = os.environ.get('Environment')
print(f"Environment: {env!r}")


Environment: 'prod'


In [2]:
%reload_ext jupyter_dmdg

In [3]:
%%athena_to_df --database f"{env}-luna-pharma-prepared-data" --out df_test 
select ar.group_id, ar.categorie, ar.code_article, ar.libelle,
    jh.job_name, jh.sw_no_lot, jh.browse_description, jh.sw_job_type, jh.job_status, 
    sa.id_numeric, sa.status as sample_status, sa.sample_type, sa.date_authorised, sa.authorisation_notes,sa.sw_conformite,
    sa.id_text,sa.sw_reac_date_liv

from sample sa 

join job_header jh on jh.job_name = sa.job_name
join sw_article ar on ar.code_article = jh.sw_article


where ar.group_id like 'ARA%'
and jh.sw_job_type ='SOG'
and jh.sw_lot_validation = 'F'
and jh.sw_no_lot not like '%FOR%'
and from_iso8601_timestamp(sa.sw_reac_date_liv) BETWEEN current_date - INTERVAL '7' day AND current_date;


In [4]:
df_test.head(5)

,group_id,categorie,code_article,libelle,job_name,sw_no_lot,browse_description,sw_job_type,job_status,id_numeric,sample_status,sample_type,date_authorised,authorisation_notes,sw_conformite,id_text,sw_reac_date_liv
0,ARAMON,PF,ARA_666341,IRBESARTAN VOIE DIRECTE,ARA_666341_2019_04,9AR0143,Job created on 19-NOV-2019 10:50 by CUERQ usin...,SOG,C,2556218,A,SOG,2023-02-03T11:50:58,ras,A,S-ARA_666341_2019_04-444,2023-02-03T11:51:44
1,ARAMON,PF,ARA_666341,IRBESARTAN VOIE DIRECTE,ARA_666341_2019_04,9AR0143,Job created on 19-NOV-2019 10:50 by CUERQ usin...,SOG,C,2556219,A,SOG,2022-12-01T10:51:40,CONFORME - Validation automatique - sw_back_csmlp,A,S-ARA_666341_2019_04-451,2023-02-03T11:51:44
2,ARAMON,PF,ARA_187859,CHLORHYDRATE AMIODARONE (PA),ARA_187859_2020_07,AAR0114,Job created on 1-DEC-2020 10:47 by CUERQ usin...,SOG,C,2566380,A,SOG,2023-02-03T16:16:32,,A,S-ARA_187859_2020_07-843,2023-02-03T16:17:45
3,ARAMON,PF,ARA_241575,NALBUPHINE CHLORHYDRATE,ARA_241575ICH_2019_01,9AR0001,Job created on 19-JUL-2019 09:28 by CUERQ usin...,SOG,V,2462784,A,SOG,2023-02-08T15:20:03,2021-CHIMIE-OOS-117,R,S-ARA_241575ICH_2019_01-795,2023-02-08T15:22:01
4,ARAMON,PF,ARA_241575,NALBUPHINE CHLORHYDRATE,ARA_241575ICH_2019_01,9AR0001,Job created on 19-JUL-2019 09:28 by CUERQ usin...,SOG,V,2462785,A,SOG,2022-12-01T14:31:53,CONFORME - Validation automatique - sw_back_csmlp,A,S-ARA_241575ICH_2019_01-805,2023-02-08T15:22:01


In [5]:
database_name = f'prod-dmdg-workspace-public-aramon-smartqc-approval-sog-20230210103512425'

In [6]:
table_name = f'{env}_ARA_luna_smartqc_Approval-SOG'

In [7]:
path = f's3://sanofi-datalake-dmdg-workspaces-prod/public-aramon-smartqc-approval-sog-20230210103512425'

In [8]:
workspace_prefix =f'public-aramon-smartqc-approval-sog-20230210103512425'

In [9]:
wr.s3.to_parquet(
    df=df_test,
    path=path,
    database=database_name,
    dataset=True,
    table=table_name,
    mode='overwrite',
    index=False,
    compression=None,
)

{'paths': ['s3://sanofi-datalake-dmdg-workspaces-prod/public-aramon-smartqc-approval-sog-20230210103512425/f3f6dbbdc9994359a6087eff9226825f.parquet'],
 'partitions_values': {}}

In [10]:
df_test.shape

(25, 17)